In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
from typing import List
import multiprocessing as mp
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
qasar_t_dir = "/content/drive/My Drive/squad/"

In [0]:
import os
print(os.listdir(qasar_t_dir))

['uncased_L-12_H-768_A-12.zip', 'train-v1.1.json', 'dev-v1.1.json', 'evaluate-v1.1.py', 'run_squad.py', 'model_repo', 'modeling.py', 'optimization.py', 'tokenization.py', 'run_squad.1.py', 'run_squad.py.gdoc', '__pycache__', 'qa_datasets_converter.git', 'qa_datasets_converter', 'quasar-t_to_squad_dev_questions_out.json', 'quasar-t_to_squad_train_questions_out.json']


### Create directory where the pre trained embedding will be downloaded

In [0]:
%cd /content/drive/My Drive/squad

/content/drive/My Drive/squad


In [0]:
!pwd

/content/drive/My Drive/squad


### download the relavent files and model from official google storage location.This will be done only once. later the code can be commented out

In [0]:
#!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [0]:
#import zipfile
#repo = 'model_repo'
#with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
#    zip_ref.extractall(repo)

In [0]:
!ls 'model_repo/uncased_L-12_H-768_A-12'

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [0]:
#!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
#!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
#!wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 
#!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [0]:
!ls model_repo/uncased_L-12_H-768_A-12

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [0]:
#!mkdir ./model_repo/output

In [0]:
import tensorflow
tensorflow.__version__

'1.15.2'

### Tensor flow should be less than 2.0. Please un comments these lines and un unstall version 2.0 or higher

In [0]:
#!pip uninstall tensorflow -y
#!pip install tensorflow==1.15.2

In [0]:
import tensorflow
tensorflow.__version__

'1.15.2'

### Run the finetuning module on Quasar train and predict on quasar dev. Save the training model and predictions inside ./model_repo/output/
#### Now go to model_repo/uncased_L-12_H-768_A-12 directory and open bert_config.json and manually change the model to new finetuned model. Some more details can be found at "https://github.com/google-research/bert#squad-11"

In [0]:
!python run_squad.1.py \
  --vocab_file=./model_repo/uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=./model_repo/uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=./model_repo/uncased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=quasar-t_to_squad_train_questions_out.json \
  --do_predict=True \
  --predict_file=quasar-t_to_squad_dev_questions_out.json \
  --train_batch_size=12 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=./model_repo/output/

#### Predictions

In [0]:
python evaluate-v1.1.py quasar-t_to_squad_dev_questions_out.json ./squad/predictions.json